In [1]:
import requests
import pandas as pd
import numpy as np
from prettytable import PrettyTable
import time

# Function to fetch trading pair data
def fetch_pair_data(pair):
    url = "https://api.dexscreener.com/latest/dex/search"
    params = {'q': pair}
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

# Function to fetch token data by token address
def fetch_token_data(chain_id, token_address):
    url = f"https://api.dexscreener.com/orders/v1/{chain_id}/{token_address}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

# Function to analyze liquidity data
def analyze_liquidity(df):
    if 'liquidity_usd' in df.columns:
        df['liquidity_usd'] = pd.to_numeric(df['liquidity_usd'], errors='coerce')
    else:
        raise KeyError("The 'liquidity_usd' column is missing in the DataFrame.")
    
    mean_liquidity = df['liquidity_usd'].mean()
    median_liquidity = df['liquidity_usd'].median()
    std_liquidity = df['liquidity_usd'].std()
    min_liquidity = df['liquidity_usd'].min()
    max_liquidity = df['liquidity_usd'].max()
    
    thresholds = [10000, 50000, 100000]
    probabilities = {f'Prob > {threshold}': np.mean(df['liquidity_usd'] > threshold) for threshold in thresholds}
    
    return {
        'mean_liquidity': mean_liquidity,
        'median_liquidity': median_liquidity,
        'std_liquidity': std_liquidity,
        'min_liquidity': min_liquidity,
        'max_liquidity': max_liquidity,
        'probabilities': probabilities
    }

# Function to analyze tick changes
def analyze_ticks(df):
    df['liquidity_usd'] = pd.to_numeric(df['liquidity_usd'], errors='coerce')
    df['prev_liquidity'] = df['liquidity_usd'].shift(1)
    df['change'] = df['liquidity_usd'] - df['prev_liquidity']
    
    upticks = df[df['change'] > 0].shape[0]
    downticks = df[df['change'] < 0].shape[0]
    no_change = df[df['change'] == 0].shape[0]
    
    total_ticks = upticks + downticks + no_change
    
    probabilities = {
        'Probability of Uptick': upticks / total_ticks if total_ticks > 0 else np.nan,
        'Probability of Downtick': downticks / total_ticks if total_ticks > 0 else np.nan,
        'Probability of No Change': no_change / total_ticks if total_ticks > 0 else np.nan
    }
    
    return probabilities

# Function to display data using PrettyTable
def display_data(df, title):
    table = PrettyTable()
    table.field_names = df.columns.tolist()
    for index, row in df.iterrows():
        table.add_row(row.tolist())
    print(f"\n{title}:")
    print(table)

# Function to display analysis results using PrettyTable
def display_analysis_results(analysis_results, title):
    liquidity_table = PrettyTable()
    liquidity_table.field_names = ["Statistic", "Value"]
    liquidity_table.add_row(["Mean Liquidity", analysis_results['mean_liquidity']])
    liquidity_table.add_row(["Median Liquidity", analysis_results['median_liquidity']])
    liquidity_table.add_row(["Standard Deviation", analysis_results['std_liquidity']])
    liquidity_table.add_row(["Min Liquidity", analysis_results['min_liquidity']])
    liquidity_table.add_row(["Max Liquidity", analysis_results['max_liquidity']])
    
    print(f"\nDescriptive Statistics for {title}:")
    print(liquidity_table)
    
    prob_table = PrettyTable()
    prob_table.field_names = ["Threshold", "Probability"]
    for threshold, prob in analysis_results['probabilities'].items():
        prob_table.add_row([threshold, prob])
    
    print(f"\nLiquidity Probabilities for {title}:")
    print(prob_table)

def display_token_analysis(token_df, token_address):
    if 'liquidity_usd' in token_df.columns:
        token_analysis = analyze_liquidity(token_df)
        display_analysis_results(token_analysis, f"Token {token_address}")
        
        tick_probabilities = analyze_ticks(token_df)
        tick_prob_table = PrettyTable()
        tick_prob_table.field_names = ["Event", "Probability"]
        for label, prob in tick_probabilities.items():
            tick_prob_table.add_row([label, prob])
        
        print(f"\nTick Probabilities for Token {token_address}:")
        print(tick_prob_table)
    else:
        print(f"\nNo liquidity data available for Token Address {token_address}.")

# Main loop with 60-second refresh
def main():
    pair = "SOL/SOL"
    token_addresses = [
        "EQtFoBFcqj6M5KWLbetkYmp9EBi33WJ6UVeLHyx4kZoW",
        # Add more token addresses as needed
    ]
    chain_id = "solana"
    
    while True:
        # Fetch and display raw data for a specific trading pair
        pair_data = fetch_pair_data(pair)
        print("\nRaw JSON Response for Pair Data:")
        print(pair_data)
        
        if 'pairs' in pair_data:
            records = []
            for item in pair_data['pairs']:
                record = {
                    'chainId': item.get('chainId'),
                    'dexId': item.get('dexId'),
                    'pairAddress': item.get('pairAddress'),
                    'baseToken': item.get('baseToken', {}).get('symbol'),
                    'quoteToken': item.get('quoteToken', {}).get('symbol'),
                    'priceNative': item.get('priceNative'),
                    'priceUsd': item.get('priceUsd'),
                    'liquidity_usd': item.get('liquidity', {}).get('usd'),
                    'fdv': item.get('fdv'),
                    'marketCap': item.get('marketCap')
                }
                records.append(record)
            
            df = pd.DataFrame(records)
            print("\nData as DataFrame for Pair Data:")
            print(df.head())
            print("\nDataFrame Columns:")
            print(df.columns)

            try:
                analysis_results = analyze_liquidity(df)
                tick_probabilities = analyze_ticks(df)
                
                display_analysis_results(analysis_results, "Pair Data")
                
                tick_prob_table = PrettyTable()
                tick_prob_table.field_names = ["Event", "Probability"]
                for label, prob in tick_probabilities.items():
                    tick_prob_table.add_row([label, prob])
                
                print("\nTick Probabilities for Pair Data:")
                print(tick_prob_table)
                
            except KeyError as e:
                print(f"Error: {e}")
        else:
            print("No 'pairs' key found in the response.")
        
        # Fetch and display raw data for each token address
        for token_address in token_addresses:
            token_data = fetch_token_data(chain_id, token_address)
            print(f"\nRaw JSON Response for Token Address {token_address}:")
            print(token_data)
            
            if isinstance(token_data, list) and len(token_data) > 0:
                # Handle cases where the data is a list of events rather than a single order
                records = []
                for token in token_data:
                    if 'type' in token and 'status' in token:
                        record = {
                            'address': token_address,
                            'type': token.get('type'),
                            'status': token.get('status'),
                            'timestamp': token.get('paymentTimestamp')
                        }
                        records.append(record)
                
                # Creating DataFrame for the current token
                df_tokens = pd.DataFrame(records)
                
                print(f"\nData as DataFrame for Token Address {token_address}:")
                print(df_tokens.head())
                print("\nDataFrame Columns:")
                print(df_tokens.columns)
                
                # Placeholder for token analysis; adjust as needed based on the available data
                display_token_analysis(df_tokens, token_address)
            else:
                print(f"No relevant data found for token {token_address}.")
        
        # Wait for 60 seconds before refreshing
        time.sleep(60)

if __name__ == "__main__":
    main()



Raw JSON Response for Pair Data:
{'schemaVersion': '1.0.0', 'pairs': [{'chainId': 'solana', 'dexId': 'raydium', 'url': 'https://dexscreener.com/solana/8pnmphuwgb1hpefym8dhgyrsaiwgp9qj98etzgtbhd8t', 'pairAddress': '8pnmpHUWgb1hPeFyM8DHgyRsaiWGp9qJ98eTzgTbhD8T', 'baseToken': {'address': 'BChiUfGBVwEs2SPf66A3YdCf9WYnWLTR2nJma1ispump', 'name': 'TOWD', 'symbol': 'TOWD'}, 'quoteToken': {'address': 'So11111111111111111111111111111111111111112', 'name': 'Wrapped SOL', 'symbol': 'SOL'}, 'priceNative': '0.0000007098', 'priceUsd': '0.00009647', 'txns': {'m5': {'buys': 20, 'sells': 5}, 'h1': {'buys': 1426, 'sells': 197}, 'h6': {'buys': 193457, 'sells': 1998}, 'h24': {'buys': 237822, 'sells': 40234}}, 'volume': {'h24': 3346954.98, 'h6': 306553.29, 'h1': 23281.8, 'm5': 845.22}, 'priceChange': {'m5': 5.31, 'h1': 20.36, 'h6': 81.88, 'h24': 36.26}, 'liquidity': {'usd': 35940.26, 'base': 186571567, 'quote': 131.9884}, 'fdv': 96480, 'marketCap': 96480, 'pairCreatedAt': 1726146012000, 'info': {'imageUrl'